In [1]:
import pandas as pd
import os

'wget' is not recognized as an internal or external command,
operable program or batch file.
unzip:  cannot find or open human+activity+recognition+using+smartphones.zip, human+activity+recognition+using+smartphones.zip.zip or human+activity+recognition+using+smartphones.zip.ZIP.


download the dataset: https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip

unzip it to this directory

In [3]:
class_map = {1:"WALKING",
2:"WALKING_UPSTAIRS",
3:"WALKING_DOWNSTAIRS",
4:"SITTING",
5:"STANDING",
6:"LAYING",}
from collections import defaultdict

In [4]:
for filename in os.listdir('test/Inertial Signals'):
    if "cleaned" in filename:
        continue
    new = []
    with open(f'test/Inertial Signals/{filename}','r') as fid:    
        for line in fid.readlines():
            new.append(line.strip().replace("  "," ").replace(" ","\t"))
            
    with open(f'test/Inertial Signals/cleaned_{filename}','w') as fid:
        for line in new:
            fid.write(line+"\n")
        
for filename in os.listdir('train/Inertial Signals'):
    if "cleaned" in filename:
        continue
    new = []
    with open(f'train/Inertial Signals/{filename}','r') as fid:    
        for line in fid.readlines():
            new.append(line.strip().replace("  "," ").replace(" ","\t"))
            
    with open(f'train/Inertial Signals/cleaned_{filename}','w') as fid:
        for line in new:
            fid.write(line+"\n")
                

In [7]:
test_dict = {}
for filename in os.listdir('test/Inertial Signals'):
    if "cleaned" not in filename:
        continue
    
    test_dict[filename] = pd.read_csv(f'test/Inertial Signals/{filename}', sep='\t', header=None).T
    
test_y = pd.read_csv("test/y_test.txt", header=None, names=['label'])
subject_df = pd.read_csv("test/subject_test.txt",  header=None, names=['subject'])
#test_y['Subject'] = subject

In [8]:
train_dict = {}
for filename in os.listdir('train/Inertial Signals'):
    if "cleaned" not in filename:
        continue
    
    train_dict[filename] = pd.read_csv(f'train/Inertial Signals/{filename}', sep='\t', header=None).T
    
train_y = pd.read_csv("train/y_train.txt", names=['label'], header=None)
train_subject_df = pd.read_csv("train/subject_train.txt",  header=None, names=['subject'])
#test_y['Subject'] = subject

In [20]:
train_dict.keys()

dict_keys(['cleaned_body_acc_x_train.txt', 'cleaned_body_acc_y_train.txt', 'cleaned_body_acc_z_train.txt', 'cleaned_body_gyro_x_train.txt', 'cleaned_body_gyro_y_train.txt', 'cleaned_body_gyro_z_train.txt', 'cleaned_total_acc_x_train.txt', 'cleaned_total_acc_y_train.txt', 'cleaned_total_acc_z_train.txt'])

In [9]:

def convert_data(data_dict, y_df, subject_df, set_type):
    dai = []
    count = defaultdict(int)
    for i in range(y_df.shape[0]):
        tmp_df=pd.DataFrame()
        for key in data_dict.keys():
            tmp_df[key.replace("cleaned_","").replace("_train.txt","").replace("_test.txt","")] = data_dict[key][i]
        
        label = class_map[y_df.loc[i][0]]    
        subject = subject_df.loc[i][0]
        count[f"{subject}_{label}"]+=1
        index = count[f"{subject}_{label}"]
        tmp_df.to_csv(f"data/{subject}_{label}_{index:02d}.csv", index=None)
        
        tmp_dai = {'file_name': f"{subject}_{label}_{index:02d}.csv",
                    "metadata":[{'name':"Subject", "value":f"{subject}"},
                                {"name": "Set", "value": set_type}],
                                "sessions":[
                                    {'session_name':"Label Session",
                                    "segments":[{"name":"Label",
                                                "value":label,
                                                "start":0,
                                                "end":127}]}
                                ]}
        
        dai.append(tmp_dai)
        
    return dai



In [10]:

def convert_data_combined(data_dict, metadata_df, set_type):
    dai = []
    count = defaultdict(int)
    groups = metadata_df.groupby(['label','subject'])
    for key in groups.groups.keys():
        M = []
        segments = []
        label = class_map[key[0]]    
        count = 0
        subject = key[1]        
        for i in groups.get_group(key).index.values:

            tmp_df=pd.DataFrame()
            #for key in data_dict.keys():
            #    tmp_df[key.replace("cleaned_","").replace("_train.txt","").replace("_test.txt","")] = data_dict[key][i]
            
            segments.append({"name":"Label","value":label,"start":count*128,"end":(count+1)*128-1})                      
            #M.append(tmp_df)
            count+=1
        
            
        
        tmp_dai = {'file_name': f"{subject}_{label}.csv",
                        "metadata":[{'name':"Subject", "value":f"{subject}"},
                                    {"name": "Set", "value": set_type}],
                                    "sessions":[
                                        {'session_name':"Label Session",
                                        "segments":segments}
                                    ]}    
        dai.append(tmp_dai)  
        #capture_df = pd.concat(M).reset_index(drop=True)
        #capture_df.to_csv(f"data/{subject}_{label}.csv", index=None)
        
    return dai



In [11]:
test_y['subject'] = subject_df
train_y['subject']= train_subject_df

dai_test = convert_data_combined(test_dict, test_y, "Test")
dai_train = convert_data_combined(train_dict, train_y, "Train")

In [12]:
import json
json.dump(dai_test+dai_train, open("data/project.dai",'w'))

: 

In [29]:
dai_test = convert_data(test_dict, test_y, subject_df)
dai_train = convert_data(train_dict, train_y, train_subject_df)

In [30]:
import json
json.dump(dai_test+dai_train, open("data/project.dai",'w'))